In [2]:
%pip install wandb
%pip install graphviz
%pip install torchviz
import wandb
wandb.login()#doesnt detect WANDB_NOTEBOOK_NAME on windows

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Python310\python.exe -m pip install --upgrade pip' command.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Note: you may need to restart the kernel to use updated packages.


wandb: Currently logged in as: tetetoutou (simclr-doctoral-research). Use `wandb login --relogin` to force relogin


True

In [3]:
wandb.init(project="test-project", entity="simclr-doctoral-research")

In [4]:
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 10,
  "batch_size": 128
}

In [5]:
import os
import wandb
import torch
import torchvision
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 10
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)  
        self.conv2 = nn.Conv2d(16, 4, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        #Decoder
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 2, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(16, 3, 2, stride=2)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.t_conv1(x))
        x = F.sigmoid(self.t_conv2(x))
        return x





In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        #img = img.view(img.size(0), -1)//makes the image 3D i think
        img = Variable(img).to(device)
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.item()))
    #if epoch % 10 == 0:
        #pic = to_img(output.cpu().data)
        #save_image(pic, './mlp_img/image_{}.png'.format(epoch))
    wandb.log({"loss": loss})

    wandb.watch(model)
    # print('epoch [{}/{}], loss:{:.4f}'
    # #       .format(epoch + 1, num_epochs, loss.data[0]))
    # if epoch % 10 == 0:
    #     pic = to_img(output.cpu().data)
    #     save_image(pic, './mlp_img/image_{}.png'.format(epoch))




C:\Python310\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
C:\Python310\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([128, 1, 28, 28])) that is different to the input size (torch.Size([128, 3, 28, 28])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Python310\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([96, 1, 28, 28])) that is different to the input size (torch.Size([96, 3, 28, 28])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch [1/10], loss:0.9282
epoch [2/10], loss:0.8771
epoch [3/10], loss:0.8668
epoch [4/10], loss:0.8703
epoch [5/10], loss:0.8594
epoch [6/10], loss:0.8577
epoch [7/10], loss:0.8665
epoch [8/10], loss:0.8618
epoch [9/10], loss:0.8709
epoch [10/10], loss:0.8536


In [7]:
from torchviz import make_dot
y = model(img)
make_dot(y, params=dict(list(model.named_parameters()))).render("torchviz_conv_autoencoder", format="png")

'torchviz.png'

In [17]:
#show the resulting image of the model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch

def show_the_model(model,image_batch):
    with torch.no_grad():
        output = model(image_batch)
        #pic = to_img(output.cpu().data)
        #if the folder doesnt exist, create it
        if not os.path.exists('./conv_entoencoder'):
            os.mkdir('./conv_entoencoder')

        save_image(output.cpu().data, './conv_entoencoder/image.png')
        img = mpimg.imread('./conv_entoencoder/image.png')
        plt.imshow(img)
        plt.show()

show_the_model(model,img)


TypeError: conv2d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)


In [ ]:
#if the directoruy does not exist, create it
if not os.path.exists('./saved_models'):
    os.makedirs('./saved_models')
torch.save(model.state_dict(), './saved_models/conv_autoencoder.pth')